# Fondamenti di elaborazione immagini

## Effettuiamo l'import delle librerie utilizzate nell'esercitazione.

In [ ]:
import cv2
import numpy as np
import matplotlib as mapli
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

Di seguito i riferimenti alle pagine di documentazione, sempre utiliti:

* Rif: [numpy](https://numpy.org/doc/stable/)
* Rif: [opencv](https://docs.opencv.org/)
* Rif: [matplotlib](https://matplotlib.org/stable/index.html)

Aggiungiamo alcune funzioni di utilita' per la scrittura del codice.

In [ ]:
def rgb(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gray(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grid(images : list[np.array], rows : int, cols : int, size : int, colors : list[str] = None) -> None:
    fig = plt.figure(figsize=(size,size))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0.1)

    if colors is not None:
        counter = 0
        for ax, im in zip(grid, images):
            ax.imshow(im, cmap=colors[counter])
            counter = (counter + 1) % len(colors)
        plt.show()
    else:
        for ax, im in zip(grid, images):
            ax.imshow(im)
        plt.show()

## _Sappiamo che le immagini sono matrici di numeri e, in quanto tali, operazioni aritmentiche elemento per elemento sono permesse e significative._

Le operazioni applicabili sono molte e combinabili fra loro percio', di seguito, mostreremo solo alcuni esempi.

Un esempio di applicazione della sottrazione lo possiamo trovare con la ricerca delle differenze. Quando, ad esempio, si vogliono realizzare architetture in cui si localizzano oggetti in movimento, diventa importante capire come da una scena all'altra avvengano i cambiamenti. Con il valore assoluto di una differenza pixel a pixel, possiamo ottenere un risultato significativo. Usiamo il metodo _absdiff_ di _opencv_.

* Rif: [absdiff](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga6fef31bc8c4071cbc114a758a2b79c14)

In [ ]:
img_a = cv2.imread('./imgs/differences/a.png')
img_b = cv2.imread('./imgs/differences/b.png')

grid([rgb(img_a), rgb(img_b)], 1, 2, 30)

Eseguiamone quindi la differenza.

In [ ]:
diff = cv2.absdiff(img_a, img_b)

grid([rgb(img_a), rgb(img_b), rgb(diff)], 1, 3, 30)

Con una semplice sottrazione, saturata ai valori del formato uint8 (0-255), non avremmo ottenuto un risultato altrettanto apprezzabile.

In [ ]:
saturated_diff = cv2.subtract(img_a, img_b)

grid([rgb(img_a), rgb(img_b), rgb(diff), rgb(saturated_diff)], 1, 4, 30)

## _Un ulteriore esempio, prassi standard della fase di preprocessamento delle immagini in un dataset e' la sottrazione con la media e la divisione per la deviazione standard._

Questo tipo di processamento, chiamato **normalizzazione** ha lo scopo di ridimensionare la scala e la varianza dei dati.

In [ ]:
apple_1 = cv2.imread('./imgs/apples/apple_1.png')
apple_2 = cv2.imread('./imgs/apples/apple_2.png')
apple_3 = cv2.imread('./imgs/apples/apple_3.png')

apples = [apple_1, apple_2, apple_3]

apples_mean = np.mean(apples, axis=(0, 1, 2))   # media rispettivamente per r, g, b
apples_std = np.std(apples, axis=(0, 1, 2))     # deviazione standard rispettivamente per r, g, b

print(f'mean: {apples_mean}, std: {apples_std}')

Ad aiutarci, in questo caso, sono stati i metodi _numpy_ chiamati _mean_ e _std_ che rispettivamente calcolano media e deviazione standard lungo le dimensioni specificate.

* Rif: [mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
* Rif: [std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)

Procediamo quindi ad eseguire la normalizzazione.

In [ ]:
norm_apple_1 = (apple_1 - apples_mean) / apples_std
norm_apple_2 = (apple_2 - apples_mean) / apples_std
norm_apple_3 = (apple_3 - apples_mean) / apples_std

apples = [rgb(apple_1), rgb(apple_2), rgb(apple_3)]

norm_apple_1 = (255 * norm_apple_1).astype(np.uint8)
norm_apple_2 = (255 * norm_apple_2).astype(np.uint8)
norm_apple_3 = (255 * norm_apple_3).astype(np.uint8)

norm_apples = [rgb(norm_apple_1), rgb(norm_apple_2), rgb(norm_apple_3)]

grid(apples, 1, 3, 30)
grid(norm_apples, 1, 3, 30)

La normalizzazione, per il deep learning, risulta essere una tecnica di preprocessamento fondamentale al fine di ottenere risultati sotto diversi aspetti:

* Si riduce la possibilita' che i dati assumano valori "troppo alti" o "troppo bassi" che, rispettivamente, potrebbero saturare o diventare ininfluenti al fine di un corretto addestramento.
* Si agevola la convergenza verso un risultato nel problema di ottimizzazione rendendo i valori indipenti dalla loro scala e centrandone la distribuzione attorno allo zero.
* Si riduce il peso computazionale dell'addestramento, riducendo di fatto il range che i valori possono assumere durante quest'ultimo.